# 🤖 AI 代理 (Agent) 架構深度解析
### 碼農顧問筆記：從 Tools 到 MCP 的底層邏輯

這份筆記總結了現代 AI 助理 (如 Claude) 如何與外部世界互動的核心概念。

## 1. 🌍 生態系：四大核心概念

我們討論了四個相輔相成的概念，它們共同構成了 Anthropic 的 AI 助理生態系：

* **1. Tools (工具 / Function Calling)**
    * **這是什麼？**：一個**通用概念**。
    * **比喻：** 讓關在黑盒子裡的大腦（LLM）擁有一部「電話」，讓它能請求（Call）外部資訊。
    * **實作：** LLM 輸出一個結構化的「工具調用請求」(JSON)，你的程式碼負責執行，再把結果送回去。

* **2. Claude Skills (Claude 技能)**
    * **這是什麼？**：Anthropic 對「Tools」的**具體 SDK 實作**。
    * **比喻：** Anthropic 官方推出的「iPhone SDK」，讓你用 `anthropic` 函式庫就能輕鬆開發 App (工具)。
    * **實作：** 使用 `anthropic.utils.function_to_tool` 將 Python 函數（包含 type hints 和 docstrings）轉換成 LLM 能理解的 JSON Schema。

* **3. MCP (Model Context Protocol)**
    * **這是什麼？**：一個**開放的標準協定**。
    * **比喻：** AI 助理的「USB-C 接口」。它定義了一個標準化的插座，讓**任何**應用程式（如 Claude 桌面版）都能安全、自動地「發現」並「使用」由**任何**伺服器（如你的 FastAPI 服務）提供的 Skills。
    * **實作：** 你寫一個 Server（例如用 FastAPI），並實作 `GET /mcp/spec`（宣告能力）和 `POST /mcp/call`（執行能力）這兩個端點。

* **4. A2A (Assistant-to-Assistant)**
    * **這是什麼？**：一個基於 MCP 的**架構願景**。
    * **比喻：** AI 助理的「自動化供應鏈」。你的個人助理 (Agent 1) 在處理任務時，能自動「委派」子任務給專業的助理 (Agent 2)，而不需要你這個人類居中協調。
    * **實作：** 一個 Manager Agent 在分析任務後，透過 MCP 呼叫多個 Worker Agent (MCP 伺服器) 來協同完成複雜工作。

---

## 2. 🚀 引擎室：`stop_reason: "tool_use"` 的真正原理

我們深入探討了「工具調用」的底層實作。**魔法發生在雲端，而非本地函式庫。**

### ❌ 迷思：本地 SDK (函式庫) 很聰明
* **猜測：** LLM 輸出一個特殊字串 (如 `@call_...`)，然後 `anthropic` 函式庫在**本地**偵測到這個字串，並「假裝」API 回傳了 `stop_reason: "tool_use"`。
* **結論：** 這個猜測是**錯誤的**。

### ✅ 真相：雲端 Inference 伺服器很聰明
* **事實：** `stop_reason: "tool_use"` 是從 Anthropic **伺服器直接傳來的原始 API 回應**。`anthropic` 函式庫只是一個信差。

**完整的底層執行緒：**

1.  **[Client]** 你的程式呼叫 `client.messages.create(..., tools=[...])`。
2.  **[Cloud-Server]** 伺服器將你的 `tools` JSON Schema **注入**到 LLM 的 System Prompt 中。(你的第一個猜測是正確的)
3.  **[Cloud-LLM]** LLM 開始逐一 token 生成文字。
4.  **💡[Aha! Moment 1]** 當 LLM 決定需要用工具時，它會輸出一個**特殊的「控制符記」(Control Token)**，例如 `</tool_use_start>`。(這就是你猜的「特殊字元」的真相)
5.  **[Cloud-Server]** Inference 伺服器 (雲端主機) **偵測**到這個控制符記，**立刻攔截 (Intercept)** 了生成流程。
6.  **[Cloud-Server]** 伺服器接著**強制 (Constrain)** LLM 的後續輸出**必須**符合你提供的 JSON Schema（這稱為 Grammar-based Sampling / 結構化生成）。
7.  **[Cloud-LLM]** LLM 在這個「緊箍咒」下，生成了 `{"name": "get_stock_price", "input": ...}`。
8.  **[Cloud-Server]** 伺服器收集完這整段 JSON，將其打包，然後**建構一個 API 回應**，這個回應的 `stop_reason` 欄位被**設為 `"tool_use"`**，`content` 欄位則包含那段 JSON。
9.  **[Client]** 你的 `anthropic` 函式庫收到這個 JSON 回應，並將其反序列化 (Deserialize) 成 Python 物件。你因此讀到了 `response.stop_reason == "tool_use"`。

---

## 3. 🔄 代理迴圈 (Agentic Loop)：LLM 如何收到結果

我們也釐清了 LLM 如何取回工具的執行結果：

* **迷思：** 在 LLM 暫停時，我們把結果「注入 (Inject)」回它正在進行的推理流程中。
* **真相：** `messages` API 是**無狀態 (Stateless)** 的。你必須發起一個**全新的 API 請求**。

**正確的 Agentic 迴圈流程：**

1.  **Request 1 (你 → AI):**
    * `messages = [{"role": "user", "content": "TSLA 股價？"}]`
    * `POST /messages` (包含 `tools` 參數)

2.  **Response 1 (AI → 你):**
    * `stop_reason: "tool_use"`
    * `content: [{"type": "tool_use", "name": "get_stock_price", ...}]`

3.  **[ 你的 Client 端執行 ]**
    * 你解析 `Response 1`，發現是 `tool_use`。
    * 你執行 `get_stock_price("TSLA")`，得到 `289.99`。
    * 你**建立**一個 `tool_result` 訊息。

4.  **Request 2 (你 → AI):**
    * 你**擴充** `messages` 列表，把**完整的上下文**都傳回去：
    ```python
    messages = [
      {"role": "user", "content": "TSLA 股價？"}, # 原始請求
      {"role": "assistant", "content": [ ... ]}, # Response 1 (AI的 tool_use 內容)
      {"role": "user", "content": [ # 你的工具結果
          {"type": "tool_result", "tool_use_id": "...", "content": "289.99"}
      ]}
    ]
    ```
    * `POST /messages` (再次發送完整的 `messages` 列表)

5.  **Response 2 (AI → 你):**
    * LLM 讀完了**完整的歷史**，現在它知道股價了。
    * `stop_reason: "end_turn"`
    * `content: [{"type": "text", "text": "TSLA 的股價目前是 289.99 元。"}]`

In [4]:
import torch

if torch.cuda.is_available():
    print("✅ Success! PyTorch can see your GPU(s).")
    device_count = torch.cuda.device_count()
    print(f"   -> Found {device_count} GPU(s).")
    for i in range(device_count):
        print(f"   -> GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("❌ Failed. PyTorch cannot see your GPU.")
    print("   -> Check NVIDIA driver, CUDA toolkit, and PyTorch installation.")

# 關鍵中的關鍵：檢查版本後綴
print(f"\nPyTorch version: {torch.__version__}")


✅ Success! PyTorch can see your GPU(s).
   -> Found 1 GPU(s).
   -> GPU 0: NVIDIA GeForce RTX 4070 Ti SUPER

PyTorch version: 2.4.1+cu121
